In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
import gc
import time

In [5]:
APIKEY='RGAPI-71daaf77-9e9e-4465-afda-7fab1b5ec490'

REGION_SET = 'kr'

ver= 5

In [6]:
#(1) request 에 대한 응답 파일을 json 형태로 변환
def call_json(fromjson): 
    return fromjson.json() 
#URL REQUEST 에 대한 RESULT로 JSON 형태, 전환해서 DATAFRAME 형태로 전환해서 RETURN

#(2) json 형태로 변환된 응답 파일을 dataframe 형태로 변환
def normalize_to_df(fromjson): 
    return json_normalize(fromjson) 
#URL REQUEST 에 대한 RESULT로 JSON 형태, 전환해서 DATAFRAME 형태로 전환해서 RETURN

#(3) URL 조합해주는 함수
def assemble_URL(query,region=REGION_SET):
    assembled_URL = "https://" + region + '.api.riotgames.com/lol/' + query + '?api_key=' + APIKEY
    return assembled_URL

#(3-1) URL 조합해주는 함수, page 가 있는 경우
def assemble_URL_page(query,page,region=REGION_SET):
    assembled_URL = "https://" + region + '.api.riotgames.com/lol/' + query + '?page=' + str(page) +  '&api_key=' + APIKEY
    return assembled_URL

#(4) URL 요청해서 응답 받는 함수
def request_query(query):
    return requests.get(query)

#(4-1) URL 요청해서 응답 받는 함수
def request_query_json(query):
    return requests.get(query).json()

#(5) SummonerId 로 accountId 구하는 함수
def ask_accountId(summonerId,region=REGION_SET):
    temp_req_df=pd.DataFrame()
    temp_accountId=request_query(assemble_URL('summoner/v4/summoners/'+summonerId,region))
    if not str(temp_accountId) == '<Response [200]>' :
        print(temp_accountId, ': problem is occured!')
        gc.collect()
        return
    else :
        temp_req_df = json_normalize(call_json(temp_accountId))['accountId'][0]
        gc.collect()
        return temp_req_df  
#(5-1) SummonerId 로 accountId 구하는 함수
def ask_accountId_json(summonerId,region=REGION_SET):
    temp_accountId=request_query_json(assemble_URL('summoner/v4/summoners/'+summonerId,region))
    try : 
        temp_req_df = temp_accountId.get('accountId')
        
    except KeyError :
        temp_req_df = 'Nan'
    gc.collect()
    return temp_req_df

#(6) ask_count 변수를 받아서 해당 변수가 1~99 (99개) 까지만 처리하고 99개 일때, sleep 2분을 걸어서 초과 요청 방지
#    query 문 이후 해당 변수값을 인자로 주고 처리된 값을 반환
def ask_count(count_var):
    count_var += 1
    if count_var == 48:
        #print('PLZ, wait a moment for sleep')
        time.sleep(10)
        count_var = 0
    return count_var

#10초에 50번 제한 10초/50번이므로 1회 요청에 0.2초 sleep
ratelimit = 0.2

def ask_ratelimit():
    time.sleep(ratelimit)

#(7) change ChampId to Name
def ChangeIDToName(id):
    switcher={
        266: "Aatrox",
        412: "Thresh",
        23: "Tryndamere",
        79: "Gragas",
        69: "Cassiopeia",
        136: "Aurelion Sol",
        13: "Ryze",
        78: "Poppy",
        14: "Sion",
        1: "Annie",
        202: "Jhin",
        43: "Karma",
        111: "Nautilus",
        240: "Kled",
        99: "Lux",
        103: "Ahri",
        2: "Olaf",
        112: "Viktor",
        34: "Anivia",
        27: "Singed",
        86: "Garen",
        127: "Lissandra",
        57: "Maokai",
        25: "Morgana",
        28: "Evelynn",
        105: "Fizz",
        74: "Heimerdinger",
        238: "Zed",
        68: "Rumble",
        82: "Mordekaiser",
        37: "Sona",
        96: "Kog'Maw",
        55: "Katarina",
        117: "Lulu",
        22: "Ashe",
        30: "Karthus",
        12: "Alistar",
        122: "Darius",
        67: "Vayne",
        110: "Varus",
        77: "Udyr",
        89: "Leona",
        126: "Jayce",
        134: "Syndra",
        80: "Pantheon",
        92: "Riven",
        121: "Kha'Zix",
        42: "Corki",
        268: "Azir",
        51: "Caitlyn",
        76: "Nidalee",
        85: "Kennen",
        3: "Galio",
        45: "Veigar",
        432: "Bard",
        150: "Gnar",
        90: "Malzahar",
        104: "Graves",
        254: "Vi",
        10: "Kayle",
        39: "Irelia",
        64: "Lee Sin",
        420: "Illaoi",
        60: "Elise",
        106: "Volibear",
        20: "Nunu",
        4: "Twisted Fate",
        24: "Jax",
        102: "Shyvana",
        429: "Kalista",
        36: "Dr. Mundo",
        427: "Ivern",
        131: "Diana",
        223: "Tahm Kench",
        63: "Brand",
        113: "Sejuani",
        8: "Vladimir",
        154: "Zac",
        421: "Rek'Sai",
        133: "Quinn",
        84: "Akali",
        163: "Taliyah",
        18: "Tristana",
        120: "Hecarim",
        15: "Sivir",
        236: "Lucian",
        107: "Rengar",
        19: "Warwick",
        72: "Skarner",
        54: "Malphite",
        157: "Yasuo",
        101: "Xerath",
        17: "Teemo",
        75: "Nasus",
        58: "Renekton",
        119: "Draven",
        35: "Shaco",
        50: "Swain",
        91: "Talon",
        40: "Janna",
        115: "Ziggs",
        245: "Ekko",
        61: "Orianna",
        114: "Fiora",
        9: "Fiddlesticks",
        31: "Cho'Gath",
        33: "Rammus",
        7: "LeBlanc",
        16: "Soraka",
        26: "Zilean",
        56: "Nocturne",
        222: "Jinx",
        83: "Yorick",
        6: "Urgot",
        203: "Kindred",
        21: "Miss Fortune",
        62: "Wukong",
        53: "Blitzcrank",
        98: "Shen",
        201: "Braum",
        5: "Xin Zhao",
        29: "Twitch",
        11: "Master Yi",
        44: "Taric",
        32: "Amumu",
        41: "Gangplank",
        48: "Trundle",
        38: "Kassadin",
        161: "Vel'Koz",
        143: "Zyra",
        267: "Nami",
        59: "Jarvan IV",
        81: "Ezreal",
        350: "Yuumi",
        145: "Kai'sa",
        518: "Neeko",
        142: "Zoe",
        498: "Xayah",
        517: "Sylas",
        141: "Kayn",
        516: "Ornn",
        555: "Pyke",
        164: "Camille",
        246: "Qiyana",
        497: "Rakan",

    }
    return switcher.get(id)

In [7]:
# ■ League 상위권 소환사들의 summonerId 를 list로 받아오기
H_Reg_summoner_ls = []
tier_league = ['challengerleagues', 'grandmasterleagues','masterleagues']
icount_ask = 0

for temp_tier in tier_league:
    temp_assemble = assemble_URL('league/v4/'+temp_tier+'/by-queue/RANKED_SOLO_5x5')
    print(temp_assemble)
    
    temp_leaguesummoner = request_query_json(temp_assemble).get('entries')
    ask_ratelimit()
    H_Reg_summoner_ls = H_Reg_summoner_ls + (list(x['summonerId'] for x in temp_leaguesummoner ))
  
print('Data call & merge are completed!')
print('Chellenger, Grand Master, Master Summoner Id count: ', len(H_Reg_summoner_ls))

#list 파일 저장
with open(str(REGION_SET)+'_'+str(tier_league)+"_H_Reg_summoner_ls_VER"+str(ver)+".txt", "w") as file:
    file.write(str(H_Reg_summoner_ls))

# ■ DIA TIER 고정
Reg_summoner_ls = []
tier_class = ['DIAMOND']
tier_lv = ['I','II','III','IV']

for tier_class_i in tier_class:
    print(tier_class_i,':Tier summonerId gathering is started!')
    for tier_lv_i in tier_lv:
        page_i = 1
        
        while(True):
            ask_ratelimit()
            temp_summonerId = ask_summoner_intier(tier_class_i, tier_lv_i, page_i)
                        
            if temp_summonerId==[]:
                break
            else :
                page_i += 1
                Reg_summoner_ls = Reg_summoner_ls + temp_summonerId     
                del temp_summonerId
                gc.collect
        print('page:', page_i, '& ', tier_lv_i,': for statement is end!')       
    print(tier_class_i,':Tier summonerId gathering is end!')
    
print('SummonerId ask loop is end!')
print('Dia summoner Id countl:', len(Reg_summoner_ls))

#list 파일 저장
with open(str(REGION_SET)+'_'+str(tier_class)+"_Reg_summoner_ls_VER"+str(ver)+".txt", "w") as file:
    file.write(str(Reg_summoner_ls))

# ■ PLA TIER 고정
P_Reg_summoner_ls = []
tier_class = ['PLATINUM']
tier_lv = ['I','II','III','IV']

for tier_class_i in tier_class:
    print(tier_class_i,':Tier summonerId gathering is started!')
    for tier_lv_i in tier_lv:
        page_i = 1
        
        while(True):
            ask_ratelimit()
            temp_summonerId = ask_summoner_intier(tier_class_i, tier_lv_i, page_i)
                        
            if temp_summonerId==[]:
                break
            else :
                page_i += 1
                P_Reg_summoner_ls = P_Reg_summoner_ls + temp_summonerId     
                del temp_summonerId
                gc.collect
        print('page:', page_i, '& ', tier_lv_i,': for statement is end!')       
    print(tier_class_i,':Tier summonerId gathering is end!')
    
print('SummonerId ask loop is end!')
print('Platinum summoner Id count:', len(P_Reg_summoner_ls))

#list 파일 저장
with open(str(REGION_SET)+'_'+str(tier_class)+"_P_Reg_summoner_ls_VER"+str(ver)+".txt", "w") as file:
    file.write(str(P_Reg_summoner_ls))

https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-71daaf77-9e9e-4465-afda-7fab1b5ec490
https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-71daaf77-9e9e-4465-afda-7fab1b5ec490


TypeError: 'NoneType' object is not iterable